In [30]:
from time import sleep

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [21]:

language_text = {'Russian':'Мы поехали на дачу',
                 'Tatar':'Без дачага киттек', 
                 'Kazakh':'Біз елге бардық', 
                 'Kyrgyz':'Биз өлкөгө жөнөдүк', 
                 'Bashqort':'Беҙ дачаға барҙыҡ',
                 'Uzbek':'Biz mamlakatga bordik'}

In [22]:
languages = ['tat', 'kaz', 'kir', 'bak', 'uzb'] # языки татарский, казахский, киргизский

In [123]:
def get_morph(text, language):
    lang_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[1]/div/select')
    lang_field.send_keys(language)
    
    text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea').clear()
    text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea')
    text_field.send_keys(text)
    
    button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[3]/div/button')
    button.click()
    sleep(1)
    
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    translation = soup.find_all('td',class_="text-left")
    morph = [word.text.split()[0] for word in translation]
    return morph

In [70]:
%%time

language = languages[0]
url = 'https://beta.apertium.org/index.eng.html#analysis?aLang=' + language + '&aQ='    
browser = webdriver.Chrome()
browser.get(url)

CPU times: user 10.6 ms, sys: 37.7 ms, total: 48.3 ms
Wall time: 10.4 s


In [68]:
%%time
for language, text in language_text.items():
    print(language)
    print(text)
    print(get_morph(text, language))

Russian
Мы поехали на дачу
['дача']
Tatar
Без дачага киттек
['кит']
Kazakh
Біз елге бардық
['бар']
Kyrgyz
Биз өлкөгө жөнөдүк
['жөнө']
Bashqort
Беҙ дачаға барҙыҡ
['бар']
Uzbek
Biz mamlakatga bordik
['bor']
CPU times: user 116 ms, sys: 9.99 ms, total: 126 ms
Wall time: 17.1 s


In [204]:
def get_morph(text, language):
    lang_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[1]/div/select')
    lang_field.send_keys(language)
    
    text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea').clear()
    text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea')
    
    text = drop_punc(text)
    text_field.send_keys(text)
    
    button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[3]/div/button')
    button.click()
    sleep(1)
    
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    translation = soup.find_all('td',class_="text-left")
    words = []
    for word in translation:
        word = word.text.split()
        for i, w in enumerate(word):
            if w == '↤':
                break
        word = word[:i]
        for w in word:
            words.append(w)
    return words

def drop_punc(word):
    punc = '''!()-[]{};:"\,<>./?@#$%^&*_~'''
    for p in punc:
        word = word.replace(p,'')
    return word

def get_tags(words, srl_verb):
    new_tags = []
    for word in words:
        for srl in srl_verb.keys(): 
            if drop_punc(word) in srl_verb[srl]:
                role = srl
                break
            else:
                role = 'O'
        new_tags.append(role)
    return new_tags

def get_new_roles(tat_sentence, morph_sentence, roles, words):
    result = {'verbs': []}
    metric = []
    for verb in roles:
        new_roles = {}
        srl_verb = {'verb':''}
        for srl in verb.keys():
#             morph = get_morph(verb[srl], language)[0]
            morph = verb[srl]
    #         Ищем слово в предложении по морфеме
            for i in range(len(morph), 0, -1):
                if morph[:i] in morph_sentence:
                    idx = morph_sentence.index(morph[:i])
                    new_roles[srl] = words[idx]
                    if srl == 'V':
                        srl_verb['verb'] = words[idx]
                    break

         
    #     Считаем сколько слов правильно перевели

        srl_verb['description'] = new_roles
        
        srl_verb['tags'] = get_tags(words, new_roles)

        metric.append(len(new_roles.keys()) / len(verb.keys()))
        result['verbs'].append(srl_verb)

    result['words'] = words
    result['metric'] = round(np.mean(metric),2)
    return result

def predict(tat_sentence, morph_sentence, roles, words, tries = 5):
    pred = None
    for i in range(tries):
        try:
            pred = get_new_roles(tat_sentence, morph_sentence, roles, words)
            break
        except:
            continue
    return pred

In [206]:
language = 'tat'
tat_sentence = 'Балалар өйдән кача һәм урманда югалып кала.'
roles = [{'V': 'качу', 'Arg(субъект перемещения)': 'балалар', 'Arg(начальная точка)': 'йорт'}, {'V': 'югалту', 'Arg(место)': 'урман'}]


morph_sentence = [get_morph(word, language)[0] for word in words]
words = tat_sentence.split()

print(predict(tat_sentence, morph_sentence, roles, words, tries = 5))

{'verbs': [{'verb': 'кача', 'description': {'V': 'кача', 'Arg(субъект перемещения)': 'Балалар'}, 'tags': ['Arg(субъект перемещения)', 'O', 'V', 'O', 'O', 'O', 'O']}, {'verb': 'югалып', 'description': {'V': 'югалып', 'Arg(место)': 'урманда'}, 'tags': ['O', 'O', 'O', 'O', 'Arg(место)', 'V', 'O']}], 'words': ['Балалар', 'өйдән', 'кача', 'һәм', 'урманда', 'югалып', 'кала.'], 'metric': 0.83}


In [207]:
language = 'tat'
tat_sentence = 'Без дачага киттек'
roles = [{'V': 'бару', 'Arg(субъект перемещения)': 'без', 'Arg(начальная точка)': 'дача'}]

words = tat_sentence.split()
morph_sentence = [get_morph(word, language)[0] for word in words]

print(predict(tat_sentence, morph_sentence, roles, words, tries = 5))

{'verbs': [{'verb': '', 'description': {'Arg(субъект перемещения)': 'Без', 'Arg(начальная точка)': 'дачага'}, 'tags': ['Arg(субъект перемещения)', 'Arg(начальная точка)', 'O']}], 'words': ['Без', 'дачага', 'киттек'], 'metric': 0.67}


In [208]:
language = 'tat'
tat_sentence = 'Әни рамны юды. Әти машинаны йөртте.'
roles = [{'V': 'юу', 'Arg(агенс)': 'әни', 'Arg(пациенс)': 'кыса'},
        {'V': 'йөртү', 'Arg(агенс)': 'әти'}]

words = tat_sentence.split()
morph_sentence = [get_morph(word, language)[0] for word in words]

print(predict(tat_sentence, morph_sentence, roles, words, tries = 5))

{'verbs': [{'verb': 'юды.', 'description': {'V': 'юды.', 'Arg(агенс)': 'Әни'}, 'tags': ['Arg(агенс)', 'O', 'V', 'O', 'O', 'O']}, {'verb': 'йөртте.', 'description': {'V': 'йөртте.', 'Arg(агенс)': 'Әти'}, 'tags': ['O', 'O', 'O', 'Arg(агенс)', 'O', 'V']}], 'words': ['Әни', 'рамны', 'юды.', 'Әти', 'машинаны', 'йөртте.'], 'metric': 0.83}
